In [11]:
import tensorflow as tf
from tensorboard.plugins import projector

import os
import json

In [2]:
%load_ext tensorboard

In [6]:
input_vocab_path = '../data/interim/preprocessed/input_vocab_index.json'
with open(input_vocab_path) as f:
    input_vocab_index = json.load(f)

In [12]:
log_dir = '../reports/embeddings/'
os.makedirs(log_dir, exist_ok=True)

In [23]:
max_vocab_size = 20000

In [24]:
with open(os.path.join(log_dir, 'metadata.tsv'), 'w') as f:
    word_index = input_vocab_index.keys()
    for word in word_index:
        f.write(f'{word}\n')

    # Fill in the rest of the labels with "unknown"
    for unknown in range(1, max_vocab_size - len(word_index)):
        f.write(f'unknown #{unknown}\n')

In [26]:
!tail ../reports/embeddings/metadata.tsv

unknown #2648
unknown #2649
unknown #2650
unknown #2651
unknown #2652
unknown #2653
unknown #2654
unknown #2655
unknown #2656
unknown #2657


In [27]:
!wc ../reports/embeddings/metadata.tsv

 19999  22660 200463 ../reports/embeddings/metadata.tsv


In [15]:

checkpoint = tf.train.Checkpoint()

In [16]:
checkpoint.restore(tf.train.latest_checkpoint(log_dir))

In [18]:
tf.train.list_variables(log_dir)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('decoder/attention_mechanism/memory_layer/kernel/.ATTRIBUTES/VARIABLE_VALUE',
  [512, 512]),
 ('decoder/attention_mechanism/memory_layer/kernel/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE',
  [512, 512]),
 ('decoder/attention_mechanism/memory_layer/kernel/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE',
  [512, 512]),
 ('decoder/decoder_rnn_cell/bias/.ATTRIBUTES/VARIABLE_VALUE', [2048]),
 ('decoder/decoder_rnn_cell/bias/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE',
  [2048]),
 ('decoder/decoder_rnn_cell/bias/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE',
  [2048]),
 ('decoder/decoder_rnn_cell/kernel/.ATTRIBUTES/VARIABLE_VALUE', [640, 2048]),
 ('decoder/decoder_rnn_cell/kernel/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE',
  [640, 2048]),
 ('decoder/decoder_rnn_cell/kernel/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE',
  [640, 2048]),
 ('decoder/decoder_rnn_cell/recurrent_kernel/.ATTRIBU

In [20]:
tf.train.load_variable(log_dir, 'encoder/embedding/embeddings/.ATTRIBUTES/VARIABLE_VALUE').shape

(20000, 128)

In [28]:
# Set up config
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`
embedding.tensor_name = 'encoder/embedding/embeddings/.ATTRIBUTES/VARIABLE_VALUE'
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

In [29]:
%tensorboard --logdir ../reports/embeddings/